<div align="center", style="color: purple">

# Zadanie 1

</div>

In [1]:
import numpy as np
import copy

In [2]:
# Tworzy losowego osobnika
def random_individual(n: int, x_min: float = -5.0, x_max: float = 5.0):
    return {
        'x': np.random.uniform(x_min, x_max, size=n),
        'o': np.random.uniform(0.1, 1.0, size=n)   # sigma
    }

# Tworzy populację
def random_population(mu: int, n: int, x_min: float = -5.0, x_max: float = 5.0):
    return np.array([random_individual(n, x_min, x_max) for _ in range(mu)], dtype=object)

# Ocena populacji
def population_evaluation(P, F):
    return np.array([F(ind['x']) for ind in P])

# Selekcja rodziców (z powtórzeniami!)
def parent_selection(P, lam: int):
    idx = np.random.choice(len(P), lam, replace=True)
    return P[idx]

# Mutacja gaussowska
def mutation(Pc, tau, tau0):
    lam = len(Pc)
    n = len(Pc[0]['x'])

    for i in range(lam):
        indiv = Pc[i]
        x = indiv['x']
        o = indiv['o']

        # Mutacja parametrów strategii
        N0 = np.random.normal()
        Ni = np.random.normal(size=n)

        o_new = o * np.exp(tau0 * N0 + tau * Ni)

        # Mutacja wartości x
        x_new = x + o_new * np.random.normal(size=n)

        indiv['x'] = x_new
        indiv['o'] = o_new

    return Pc

def replacement_plus(P, Pc, mu, F):
    combined = np.array([copy.deepcopy(ind) for ind in list(P) + list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

# Replacement: ES(mu, lambda)
def replacement_comma(Pc, mu, F):
    combined = np.array([copy.deepcopy(ind) for ind in list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

In [3]:
ITER = 10000

# Strategia ewolucyjna ES(mu + lambda)
def ES_plus(mu, lam, tau, tau0, F, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0)
        P = replacement_plus(P, Pc, mu, F)

    fitness = population_evaluation(P, F)
    return P[np.argmax(fitness)]

# Strategia ewolucyjna ES(mu, lambda)
def ES_comma(mu, lam, tau, tau0, F, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0)
        P = replacement_comma(Pc, mu, F)

    fitness = population_evaluation(P, F)
    return P[np.argmax(fitness)]

## Problemy bez ograniczeń

In [4]:
def sphere(x):
    return -np.sum(x**2)  # minimalizacja -> dajemy minus

D = 100
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=sphere, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-sphere(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=sphere, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-sphere(best)}')

Najlepszy osobnik: [ 4.40046919e-04 -1.75877502e-02  1.60395351e-02  1.60054076e-02
  2.01223646e-02  1.92511807e-03 -3.93407265e-03  8.28237830e-03
 -1.78682472e-01 -8.52387277e-03  5.59490207e-03  1.50027468e-02
  8.11470651e-03  5.84146319e-03  6.47831342e-03 -4.48013544e-03
 -1.04748901e-02  9.00240960e-03 -1.82854044e-03 -1.08239740e-02
  4.08179374e-03  1.38918639e-01  1.38617359e-01  4.67864674e-03
  9.48751970e-03 -7.00163228e-03 -4.93307206e-03  7.89260278e-04
 -2.81008423e-03  9.41258388e-03  1.05553683e-03  1.26450595e-02
 -2.82271606e-04 -6.75964320e-03 -1.83525550e-04  7.62544696e-03
  1.00141358e-02  5.07524924e-03  5.59031455e-03  6.06290406e-01
  1.90421068e-02 -8.56481215e-03  3.24337680e-03  2.12676130e-01
  1.10543275e-01  1.57280995e-03  7.13565333e-03 -4.76903309e-03
 -6.76741649e-03 -1.04349251e-02  6.93876606e-03  8.26664845e-03
  1.02983552e+00 -5.40972992e-03 -6.39902697e-03  8.49376118e-03
  1.54119770e-02  6.46887723e-03  7.99448951e-03 -4.05464028e-04
 -4.14

In [5]:
def easom(x):
    x1, x2 = x[0], x[1]
    return np.cos(x1) * np.cos(x2) * np.exp(-((x1 - np.pi)**2 + (x2 - np.pi)**2))

D = 2
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=easom, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-easom(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=easom, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-easom(best)}')

Najlepszy osobnik: [3.14159265 3.14159265]
Wartość, którą osiąga: -1.0

Najlepszy osobnik: [3.14159265 3.14159265]
Wartość, którą osiąga: -1.0


In [6]:
def dixon_price(x):
    x = np.array(x)
    term1 = (x[0] - 1)**2
    term2 = np.sum(np.arange(2, len(x)+1) * (2*x[1:]**2 - x[:-1])**2)
    return -(term1 + term2)

D = 100
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=dixon_price, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-dixon_price(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=dixon_price, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-dixon_price(best)}')

Najlepszy osobnik: [-1.05562092e+00 -5.41843084e-03  2.28560254e-01 -4.10918570e-01
 -4.56000887e-02 -2.80047379e-03  2.07648106e-02  2.42873199e-01
  3.64988318e-01  4.62508117e-01 -5.16275120e-01  9.04680008e-04
  1.13790724e-01 -9.93615224e-02  5.17206485e-02  2.87242396e-01
  4.20506050e-01  5.20604433e-01  6.03300869e-01  7.51820255e-01
 -5.00193062e-01  2.01229529e-03 -2.15262443e-03 -5.08713611e-02
  2.38851636e-02 -1.37229145e-01  4.14483030e-04  1.30817894e-01
  2.72107256e-01  4.23991653e-01  5.23347548e-01  5.04819605e-01
  5.10853946e-01  5.08025579e-01  5.14995247e-01  5.14609333e-01
  5.25208801e-01  5.46484206e-01  6.01500269e-01  5.96673410e-01
  5.43216579e-01  4.66183546e-01 -3.62641349e-01  6.90015214e-02
  3.03334541e-01  4.20673084e-01  4.52761786e-01  4.73485857e-01
  4.77511983e-01  4.83611567e-01  4.93307973e-01  4.99456364e-01
  5.07329245e-01  5.11196788e-01  5.22275572e-01  5.51535373e-01
  5.95074422e-01  5.41915746e-01  5.10396571e-01  4.84076886e-01
  4.43

In [7]:
def rastrigin(x):
    x = np.array(x)
    n = len(x)
    return -(10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x)))

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=rastrigin, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-rastrigin(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=rastrigin, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-rastrigin(best)}')

Najlepszy osobnik: [ 9.95355761e-01  9.94519302e-01 -9.94904729e-01 -1.98987393e+00
 -9.94955300e-01  1.98990598e+00 -1.98978285e+00 -2.98483680e+00
 -9.94976457e-01  1.98970859e+00  9.95011633e-01  9.95078382e-01
 -1.98991675e+00 -2.98484358e+00 -2.98498443e+00  2.98493220e+00
  1.07425252e-05 -1.98991049e+00  1.26194179e-05  8.42818168e-04]
Wartość, którą osiąga: 66.66233643333516

Najlepszy osobnik: [ 2.98486385e+00 -2.98485606e+00  9.94959813e-01 -2.98486519e+00
 -1.98988385e+00  1.98991215e+00  1.87306498e-06  9.94958336e-01
  9.95226757e-01 -2.98496254e+00  2.97750951e+00  1.16053187e-05
 -9.94960353e-01  1.99027033e+00  3.93644812e-07 -1.98991576e+00
 -1.98991285e+00  2.98485546e+00  4.82174168e-07 -9.94958491e-01]
Wartość, którą osiąga: 78.6122417023873


In [8]:
def schwefel(x):
    x = np.array(x)
    n = len(x)
    return -(418.9829 * n - np.sum(x * np.sin(np.sqrt(np.abs(x)))))

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=schwefel, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-schwefel(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=schwefel, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-schwefel(best)}')

Najlepszy osobnik: [  5.23919849   5.23919916   5.23919856 -25.87741705   5.23919823
   5.23920041   5.23919934   5.23919943 -25.87741786   5.23919812
   5.23919955   5.23919935   5.23919866   5.23919817   5.23920105
   5.23919916   5.23919812   5.23919805   5.23919926   5.23919723]
Wartość, którą osiąga: 8260.476650298748

Najlepszy osobnik: [  5.23920966   5.23919946   5.2391987    5.23824505   5.23920751
   5.2391986    5.23920116   5.23919954   5.23919917   5.23920023
   5.23919919   5.23920132   5.23920092   5.23920195   5.23921867
 -25.87741778   5.23919969   5.23919203   5.23919935   5.23920009]
Wartość, którą osiąga: 8280.614309080529


In [9]:
def griewank(x):
    x = np.array(x)
    n = len(x)

    sum_term = np.sum(x**2) / 4000
    prod_term = np.prod(np.cos(x / np.sqrt(np.arange(1, n+1))))

    return -(1 + sum_term - prod_term)

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=griewank, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-griewank(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=griewank, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-griewank(best)}')

Najlepszy osobnik: [ 3.91427590e-06  7.54012263e-06 -8.86346991e-06 -5.18934164e-06
  4.95807119e-05  9.97067175e-06 -2.04602631e-05  2.59165629e-05
  6.22259355e-06 -3.26261816e-05  2.91937985e-05 -2.14011132e-05
  7.52833159e-05  2.87160562e-06 -1.39315966e-03  6.24429975e-05
 -3.81221927e-05 -2.38823214e-06 -1.04540225e-06 -7.36382937e-06]
Wartość, którą osiąga: 6.604795155240595e-08

Najlepszy osobnik: [ 3.14027442e+00  1.05250643e-04  6.62269463e-06 -2.82713101e-06
 -5.22025501e-06  7.67228374e+00  2.66421552e-04 -8.34282776e-06
 -2.57433476e-05 -1.94168469e-05  3.21397735e-06  3.43807341e-06
 -1.35725414e-05 -1.19566242e-05  1.14674147e-05 -1.43569575e-05
 -4.13981663e-05 -5.31501801e-05  4.55489241e-06  4.10884223e-05]
Wartość, którą osiąga: 0.017226333716272912


# Problemy z ograniczeniami

In [10]:
import numpy as np
import copy

In [11]:
# Tworzy losowego osobnika
def random_individual(n: int, x_min: float = -5.0, x_max: float = 5.0):
    return {
        'x': np.random.uniform(x_min, x_max, size=n),
        'o': np.random.uniform(0.1, 1.0, size=n)   # sigma
    }

# Tworzy populację
def random_population(mu: int, n: int, x_min: float = -5.0, x_max: float = 5.0):
    return np.array([random_individual(n, x_min, x_max) for _ in range(mu)], dtype=object)

# Ocena populacji
def population_evaluation(P, F, G):
    if G:
        return np.array([F(ind['x']) - G(ind['x']) for ind in P])
    return np.array([F(ind['x']) for ind in P])

# Selekcja rodziców (z powtórzeniami!)
def parent_selection(P, lam: int):
    idx = np.random.choice(len(P), lam, replace=True)
    return P[idx]

# Mutacja gaussowska
def mutation(Pc, tau, tau0, low, high):
    lam = len(Pc)
    n = len(Pc[0]['x'])

    for i in range(lam):
        indiv = Pc[i]
        x = indiv['x']
        o = indiv['o']

        # Mutacja parametrów strategii
        N0 = np.random.normal()
        Ni = np.random.normal(size=n)

        o_new = o * np.exp(tau0 * N0 + tau * Ni)

        # Mutacja wartości x
        x_new = x + o_new * np.random.normal(size=n)
        x_new = np.clip(x_new, low, high)

        indiv['x'] = x_new
        indiv['o'] = o_new

    return Pc

def replacement_plus(P, Pc, mu, F, G):
    combined = np.array([copy.deepcopy(ind) for ind in list(P) + list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F, G)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

# Replacement: ES(mu, lambda)
def replacement_comma(Pc, mu, F, G):
    combined = np.array([copy.deepcopy(ind) for ind in list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F, G)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

In [12]:
ITER = 10000

# Strategia ewolucyjna ES(mu + lambda)
def ES_plus(mu, lam, tau, tau0, F, G, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0, x_min, x_max)
        P = replacement_plus(P, Pc, mu, F, G)

    fitness = population_evaluation(P, F, G)
    return P[np.argmax(fitness)]

# Strategia ewolucyjna ES(mu, lambda)
def ES_comma(mu, lam, tau, tau0, F, G, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0, x_min, x_max)
        P = replacement_comma(Pc, mu, F, G)

    fitness = population_evaluation(P, F, G)
    return P[np.argmax(fitness)]

In [13]:
# --------------------------
# Funkcja celu G2 (maksymalizacja)
# --------------------------
def g2_objective(x):
    x = np.array(x)
    n = len(x)

    numerator = np.sum(np.cos(x)**4) - 2 * np.prod(np.cos(x)**2)
    denominator = np.sqrt(np.sum((np.arange(1, n+1) * x**2)))

    return numerator / denominator


# --------------------------
# Ograniczenia (<= 0)
# --------------------------

def g2_constraint_1(x):
    # g1(x) = Π xi  + 0.75 <= 0
    return -np.prod(x) + 0.75


def g2_constraint_2(x):
    # g2(x) = Σ xi - 7.5n <= 0
    x = np.array(x)
    n = len(x)
    return np.sum(x) - 7.5 * n

def g_penalty(x):
    penalty = 0

    c1 = g2_constraint_1(x)
    c2 = g2_constraint_2(x)

    if c1 > 0:
        penalty += 1000000

    if c2 > 0:
        penalty += 1000000

    return penalty

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g2_objective,  G=g_penalty, n=D, x_min=0.0, x_max=10.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {g2_objective(best)}\n')
print(g2_constraint_1(best))
print(g2_constraint_2(best))

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g2_objective, G=g_penalty, n=D, x_min=0.0, x_max=10.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {g2_objective(best)}')
print(g2_constraint_1(best))
print(g2_constraint_2(best))


Najlepszy osobnik: [9.41565251e+00 1.58055465e+00 6.26353309e+00 8.64920105e-03
 6.25302405e+00 3.12272109e+00 9.36025388e+00 3.11732777e+00
 6.22375078e+00 3.11110528e+00 6.21664133e+00 9.41259446e-03
 3.10170136e+00 3.09815540e+00 9.17388678e-03 9.60654321e-03
 3.09089043e+00 9.66540094e-03 9.24431310e+00 6.16141808e+00]
Wartość, którą osiąga: 0.2713760301131011

-3.455116765582744e-06
-70.59244956164824
Najlepszy osobnik: [6.27801325e+00 3.13642739e+00 3.13432569e+00 9.37864509e+00
 9.38548996e+00 3.12604081e+00 3.12335988e+00 3.12078844e+00
 6.23466860e+00 6.23092552e+00 6.22569568e+00 6.22044650e+00
 6.21493404e+00 3.10553798e+00 3.10277284e+00 3.10000983e+00
 9.28898004e+00 3.09500074e+00 4.63507664e+00 3.38881985e-05]
Wartość, którą osiąga: 0.25033830788899714
-208773622.87862682
-51.86282719012996


In [21]:
# Funkcja celu G3 (maksymalizacja)
def g6_objective(x):
    x = np.array(x)
    n = len(x)
    return -(np.pow(x[0] - 10, 3) + np.pow(x[1] + 20, 3))

# Ograniczenie równościowe h1(x) = 0
def g6_constraint_1(x):
    x = np.array(x)
    return np.pow(x[0] - 5, 2) + np.pow(x[1] - 5, 2) + 100

def g6_constraint_2(x):
    x = np.array(x)
    return np.pow(x[0] - 5, 2) + np.pow(x[1] - 5, 2) + 82.81

def g_penalty(x):
    penalty = 0

    c1 = g6_constraint_1(x)

    if c1 > 0:
        penalty = 1000000

    return penalty

D = 2
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g6_objective,  G=g_penalty, n=D, x_min=0.0, x_max=100.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-g6_objective(best)}\n')
print(g6_constraint_1(best))

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g6_objective, G=g_penalty, n=D, x_min=0.0, x_max=100.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-g6_objective(best)}')
print(g6_constraint_1(best))


Najlepszy osobnik: [9.98680596e-14 1.25261256e-15]
Wartość, którą osiąga: 7000.00000000003

149.999999999999
Najlepszy osobnik: [4.40235817e-14 9.88053730e-15]
Wartość, którą osiąga: 7000.000000000026
149.99999999999946
